In [ ]:
import pandas as pd
import xlrd
import xlwt
from xlwt import Workbook
import nltk

#Use file ‘C_PigbusinessDN1.xlsx’ with the stopwords of 'farm(er)'
exl_file = '/Users/raphael/Desktop/FaceBook Data/CleanText_Varkens++.xls'

df = pd.read_excel(exl_file, sheet_name='Sheet1', nrows=486)
print(df.shape)
df = df.dropna(subset = ['CleanText'])
print(df.shape)

from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=5, token_pattern='\w+|$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(df['CleanText'])
tf = tf.toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out()
print(tf_feature_names)




In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import gensim
import gensim
from gensim.models import CoherenceModel

from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus
from nltk.tokenize import word_tokenize
import tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
import numpy as np

df['tokens'] = df['CleanText'].apply(lambda x: word_tokenize(x))
PerplexityList = []
LogLikelihoodList = []
CoherenceList = []
doc_tokens = df['tokens'].tolist()
flat_tokens = [word for doc in doc_tokens for word in doc]
flat_docs = [' '.join(doc) for doc in doc_tokens]
tf = vectorizer.fit_transform(flat_docs)
corpus = Sparse2Corpus(tf, documents_columns=False)


def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)


for number_of_topics in range(1,40,1):
    model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0, doc_topic_prior=1/number_of_topics)
    model.fit(tf)
    PerplexityList.append(model.perplexity(tf))
    #the mean accuracy on the given test data and labels
    LogLikelihoodList.append(model.score(tf))
    no_top_words = 10
    CoherenceList = metric_coherence_gensim(measure='c_v', topic_word_distrib=model.components_, dtm=corpus, vocab=np.array(tf_feature_names),texts=df['tokens'])



In [ ]:
a = PerplexityList
b = LogLikelihoodList
c = CoherenceList

In [ ]:
import matplotlib.pyplot as plt   
import numpy as np

fig = plt.figure(1)
ax = fig.add_subplot(1,1,1)
ax.plot(range(1,40,1), a, color='green', label='Perplexity')
ax.set_xlabel("# of topics")
ax.set_ylabel("Perplexity score")
plt.grid(True)
#plt.savefig('/Users/raphael/Desktop/FaceBook Data/topic modellig/Topic_Varkens/Clean++/C_Perplexity_DN14.jpg')
plt.show()


fig = plt.figure(1)
ax = fig.add_subplot(1,1,1)
ax.plot(range(1,40,1), b, color='blue', label='LogLikelihood')
ax.set_xlabel("# of topics")
ax.set_ylabel("Log-likelihood score")
plt.grid(True)
#plt.savefig('/Users/raphael/Desktop/FaceBook Data/topic modellig/Topic_Varkens/Clean++/C_Log_Likelihood_DN14.jpg')
plt.show()


fig = plt.figure(1)
ax = fig.add_subplot(1,1,1)
ax.plot(range(1,40,1), c, color='blue', label='Coherence')
ax.set_xlabel("# of topics")
ax.set_ylabel("Coherence score")
plt.grid(True)
#plt.savefig('/Users/raphael/Desktop/FaceBook Data/topic modellig/Topic_Varkens/Clean++/C_Coherence_DN14.jpg')
plt.show()